In [1]:
import pandas as pd
import numpy as np
import difflib 
from fuzzywuzzy import fuzz

In [2]:
capiq_companies = pd.read_excel('/Users/mattmullens/Desktop/Capiq/All Companies for Strategics Matching_website.xls'
                   , sheetname=0 )

In [3]:
capiq_companies['company_name_mod'] = capiq_companies['company_name']

capiq_companies['company_name_mod'] = capiq_companies['company_name_mod'].str.replace(r"\(.*\)","")
capiq_companies['company_name_mod'] = capiq_companies['company_name_mod'].str.replace( ', Inc\.',"")
capiq_companies['company_name_mod'] = capiq_companies['company_name_mod'].str.replace( ', LLC',"")
capiq_companies['company_name_mod'] = capiq_companies['company_name_mod'].str.replace( 'Inc\.',"")
capiq_companies['company_name_mod'] = capiq_companies['company_name_mod'].str.replace( 'Corp\.',"")
capiq_companies['company_name_mod'] = capiq_companies['company_name_mod'].str.replace( 'Corporation',"")
capiq_companies['company_name_mod'] = capiq_companies['company_name_mod'].str.replace( 'Group Holdings',"")
capiq_companies['company_name_mod'] = capiq_companies['company_name_mod'].str.replace( 'Company',"")
capiq_companies['company_name_mod'] = capiq_companies['company_name_mod'].str.replace( 'Co\.',"")
capiq_companies['company_name_mod'] = capiq_companies['company_name_mod'].str.replace( 'PLC',"")

In [4]:
capiq_companies['company_name_mod'] = capiq_companies['company_name_mod'].apply(lambda x: x.strip())

capiq_companies['company_name_mod'] = capiq_companies['company_name_mod'].apply(lambda x: x.upper())

In [124]:
capiq_companies.head()

,company_name,Ticker,IndustryClassifications,Primary_Industry,Primary_Sector,Geographic_Region,Headquarters_Country,State_Region,Market_Cap,Total_Rev,Gross_Profit,Website,company_name_mod,Join
0,Žitnjak proizvodnja i promet robom d.d. (ZGSE:...,ZGSE:ZTNJ-R-A,Agricultural Products (Primary); Consumer Stap...,Agricultural Products,Consumer Staples,Europe,Croatia,-,7.08,5.62,0.791,www.zitnjak.hr,ŽITNJAK PROIZVODNJA I PROMET ROBOM D.D.,1
1,01 Communique Laboratory Inc. (TSXV:ONE),TSXV:ONE,Application Software (Primary); Customer Relat...,Application Software,Information Technology,United States and Canada,Canada,Ontario,2.83,0.06,0.06,www.01com.com,01 COMMUNIQUE LABORATORY,1
2,01CYBERATON S.A. (WSE:01C),WSE:01C,Electric Power By Solar Energy (Primary); Elec...,Renewable Electricity,Utilities,Europe,Poland,-,2.07,1.59,0.805,www.01cyberaton.eu,01CYBERATON S.A.,1
3,1-800-Flowers.com Inc. (NasdaqGS:FLWS),NasdaqGS:FLWS,Consumer Discretionary (Primary); Internet and...,Internet Retail,Consumer Discretionary,United States and Canada,United States,New York,484.20,1166.9,514.9,www.1800flowers.com,1-800-FLOWERS.COM,1
4,1-PAGE LIMITED (ASX:1PG),ASX:1PG,Application Software (Primary); Enterprise Res...,Application Software,Information Technology,United States and Canada,United States,California,83.10,0.292,-10.3,www.1-page.com,1-PAGE LIMITED,1


In [123]:
capiq_companies[['Primary_Industry','Primary_Sector']].drop_duplicates().to_excel('PrimaryIndustry_Sector.xlsx',index=False)

In [6]:
len(capiq_companies)

46505

In [7]:
Funded_Companies_pre = pd.read_excel('/Users/mattmullens/Desktop/Python/Crunchbase/crunchbase_export.xlsx' , sheetname=1 , parse_cols ="A:K,U")

In [8]:
# Stupid fix for company name with mixed int/str and unicode

s = Funded_Companies_pre.company_name.str.split("|||").apply(pd.Series, 1).stack()
s.index = s.index.droplevel(-1)

In [9]:
s = s.to_frame()
s.columns=['company_name']

In [10]:
del Funded_Companies_pre['company_name']

In [11]:
# Merge new company name back on

Funded_Companies = pd.merge(Funded_Companies_pre, s, left_index=True, right_index=True).reset_index(drop=True)

In [12]:
Funded_Companies.head()

,domain,country_code,state_code,region,city,status,short_description,category_list,funding_rounds,funding_total_usd,uuid,company_name
0,watan-group.com,AFG,NaN,Kabul,Kabul,operating,Watan Group is a company based in Afghanistan ...,Logistics|Oil and Gas|Security,0,NaN,be873392-bd02-4b74-5d27-04474fc481f4,Watan Group
1,hemp.al,ALB,NaN,Tirana,Tirana,operating,Hemp Farming and CBD Hemp Oil Production,Agriculture|Health Care,1,NaN,9ba4a16f-56f2-11b3-ed34-227a62da9eeb,American-Albanian Hemp Company
2,baboon.al,ALB,NaN,Tirana,Tirana,operating,Mobile Delivery Food,Delivery|Hospitality,2,2.767441e+05,b381a20f-7f4a-59b2-5937-cc1761461004,Baboon
3,dentem.co,ALB,NaN,Tirana,Tiranë,operating,Dentem - All things dental online - Online den...,Dental|Health Care|Information Technology|Inte...,1,3.782775e+04,17c6e8c4-7e41-32ef-f255-ffff576b433c,Dentem
4,noa.com.al,ALB,NaN,Tirana,Tirana,closed,NOA SHA is a Holdings company staffed by a sma...,Financial Services|FinTech,1,7.931354e+06,ea85729f-0174-608c-8bd1-425b79f99ba8,NOA SHA


In [13]:
capiq_companies.head()

,company_name,Ticker,IndustryClassifications,Primary_Industry,Primary_Sector,Geographic_Region,Headquarters_Country,State_Region,Market_Cap,Total_Rev,Gross_Profit,Website,company_name_mod
0,Žitnjak proizvodnja i promet robom d.d. (ZGSE:...,ZGSE:ZTNJ-R-A,Agricultural Products (Primary); Consumer Stap...,Agricultural Products,Consumer Staples,Europe,Croatia,-,7.08,5.62,0.791,www.zitnjak.hr,ŽITNJAK PROIZVODNJA I PROMET ROBOM D.D.
1,01 Communique Laboratory Inc. (TSXV:ONE),TSXV:ONE,Application Software (Primary); Customer Relat...,Application Software,Information Technology,United States and Canada,Canada,Ontario,2.83,0.06,0.06,www.01com.com,01 COMMUNIQUE LABORATORY
2,01CYBERATON S.A. (WSE:01C),WSE:01C,Electric Power By Solar Energy (Primary); Elec...,Renewable Electricity,Utilities,Europe,Poland,-,2.07,1.59,0.805,www.01cyberaton.eu,01CYBERATON S.A.
3,1-800-Flowers.com Inc. (NasdaqGS:FLWS),NasdaqGS:FLWS,Consumer Discretionary (Primary); Internet and...,Internet Retail,Consumer Discretionary,United States and Canada,United States,New York,484.20,1166.9,514.9,www.1800flowers.com,1-800-FLOWERS.COM
4,1-PAGE LIMITED (ASX:1PG),ASX:1PG,Application Software (Primary); Enterprise Res...,Application Software,Information Technology,United States and Canada,United States,California,83.10,0.292,-10.3,www.1-page.com,1-PAGE LIMITED


In [14]:
Funded_Companies['company_name_mod'] = Funded_Companies['company_name']

Funded_Companies['company_name_mod'] = Funded_Companies['company_name_mod'].str.replace( ', Inc\.',"")
Funded_Companies['company_name_mod'] = Funded_Companies['company_name_mod'].str.replace( ', LLC',"")
Funded_Companies['company_name_mod'] = Funded_Companies['company_name_mod'].str.replace( 'Inc\.',"")
Funded_Companies['company_name_mod'] = Funded_Companies['company_name_mod'].str.replace( 'Corp\.',"")
Funded_Companies['company_name_mod'] = Funded_Companies['company_name_mod'].str.replace( 'Corporation',"")
Funded_Companies['company_name_mod'] = Funded_Companies['company_name_mod'].str.replace( 'Group Holdings',"")
Funded_Companies['company_name_mod'] = Funded_Companies['company_name_mod'].str.replace( 'Company',"")
Funded_Companies['company_name_mod'] = Funded_Companies['company_name_mod'].str.replace( 'Co\.',"")
Funded_Companies['company_name_mod'] = Funded_Companies['company_name_mod'].str.replace( 'PLC',"")

In [15]:
Funded_Companies['company_name_mod'] = Funded_Companies['company_name_mod'].apply(lambda x: x.strip())

Funded_Companies['company_name_mod'] = Funded_Companies['company_name_mod'].apply(lambda x: x.upper())

In [16]:
Merged = pd.merge(Funded_Companies,capiq_companies[['company_name','company_name_mod','Headquarters_Country','State_Region','Market_Cap','IndustryClassifications']], left_on='company_name_mod',right_on='company_name_mod').sort_values('company_name_mod')

In [27]:
Merged.rename(columns={'company_name_x':'company_name_CB','company_name_y':'company_name_CIQ','company_name_mod':'company_name_normalized'}, inplace=True)

In [28]:
Merged_Website = pd.merge(Funded_Companies,capiq_companies[['company_name','Headquarters_Country','State_Region','Market_Cap','Website','IndustryClassifications']], left_on='domain',right_on='Website').sort_values('company_name_x')

In [29]:
Merged.tail(30)

,domain,country_code,state_code,region,city,status,short_description,category_list,funding_rounds,funding_total_usd,uuid,company_name_CB,company_name_normalized,company_name_CIQ,Headquarters_Country,State_Region,Market_Cap,IndustryClassifications
3502,zaisfinancial.com,USA,NJ,Newark,Red Bank,ipo,ZAIS Financial Corp. (ZFC) is a externally-man...,Finance,0,NaN,150d0606-8811-aec0-97e8-a6b60f2097fc,Zais Financial,ZAIS FINANCIAL,ZAIS Financial Corp. (NYSE:ZFC),United States,New Jersey,113.900,Financials (Primary); Mortgage REITs (Primary)...
341,zalando.com,DEU,NaN,Berlin,Berlin,ipo,"Zalando is a fashion and technology company, c...",E-Commerce|Fashion|Lifestyle,5,2.399228e+08,0a9eae4d-a269-646c-0cad-cf5b559b74b6,Zalando SE,ZALANDO SE,Zalando SE (XTRA:ZAL),Germany,Berlin,7737.000,Consumer Discretionary (Primary); Internet and...
3503,zapworld.com,USA,CA,Napa Valley,Santa Rosa,operating,ZAP creates and distributes advanced technolog...,Automotive,1,2.500000e+07,47bf8409-638f-f669-6a56-63f9187ef96f,ZAP,ZAP,ZAP (OTCPK:ZAAP),United States,California,28.200,Automobile Manufacturers (Primary); Automobile...
3504,zayo.com,USA,CO,Denver,Boulder,ipo,Zayo is an international provider of dark fibe...,Infrastructure|Internet|Mobile|Telecommunications,2,3.100000e+08,7765f9aa-c89a-b47a-5523-7e41d29e4619,Zayo,ZAYO,"Zayo Group Holdings, Inc. (NYSE:ZAYO)",United States,Colorado,6697.600,Alternative Carriers (Primary); Broadband Tele...
3505,zazaenergy.com,USA,TX,Houston,Houston,ipo,ZaZa Energy Corporation is an independent expl...,Energy|Oil and Gas,0,NaN,a7d154a0-54b0-ecc2-9081-5b726c7c9174,ZaZa Energy,ZAZA ENERGY,ZaZa Energy Corporation (OTCPK:ZAZA),United States,Texas,1.350,Energy (Primary); Energy (Primary); Oil and Ga...
3506,zebra.com,USA,IL,Chicago,Lincolnshire,ipo,Asset Tracking Software,Enterprise Software,1,2.000000e+09,350f3ee6-b803-be64-7191-dc4ecbe63acd,Zebra Technologies,ZEBRA TECHNOLOGIES,Zebra Technologies Corporation (NasdaqGS:ZBRA),United States,Illinois,2611.500,Electronic Equipment and Instruments (Primary)...
3507,zeltiq.com,USA,CA,SF Bay Area,Pleasanton,ipo,Zeltiq Aesthetics is a science-based medical d...,Biotechnology,3,7.042420e+07,ee07b065-80d0-d340-2cff-2b031ca42d73,Zeltiq Aesthetics,ZELTIQ AESTHETICS,"ZELTIQ Aesthetics, Inc. (NasdaqGS:ZLTQ)",United States,California,959.500,Healthcare (Primary); Healthcare Equipment (Pr...
3508,zendesk.com,USA,CA,SF Bay Area,San Francisco,ipo,Zendesk is a software development firm providi...,Customer Service|Enterprise Software|SaaS,6,8.550000e+07,08188951-6e48-e08a-4569-f90436c594e3,Zendesk,ZENDESK,"Zendesk, Inc. (NYSE:ZEN)",United States,California,2164.200,Application Software (Primary); Customer Relat...
3509,zenoviaexchange.com,USA,GA,Atlanta,Alpharetta,operating,Zenovia Digital Exchange is a display media ex...,Advertising,2,1.104000e+07,00244ad2-e3d9-0b2d-05cc-09e12d63c77e,Zenovia Digital Exchange,ZENOVIA DIGITAL EXCHANGE,Zenovia Digital Exchange Corporation (OTCPK:ZDEC),United States,Virginia,2.360,Information Technology (Primary); Internet Sof...
3510,zntr.com,USA,AZ,Phoenix,Scottsdale,operating,"Zentric, Inc. is a technology company that bel...",EdTech|Education,1,NaN,662ae3e1-8d0b-b0b4-f67a-7fc07532bc5f,Zentric,ZENTRIC,"Zentric, Inc. (OTCPK:ZNTR)",Canada,Ontario,0.086,Batteries (Primary); Capital Goods (Primary); ...


In [30]:
len(Merged)

3654

In [31]:
len(Merged_Website)

62

In [33]:
capiq_companies[ capiq_companies['company_name'].str.contains("CO\.") ]

,company_name,Ticker,IndustryClassifications,Primary_Industry,Primary_Sector,Geographic_Region,Headquarters_Country,State_Region,Market_Cap,Total_Rev,Gross_Profit,Website,company_name_mod
1357,Al-Amal Financial Investments CO. (ASE:AMAL),ASE:AMAL,Asset Management and Custody Banks (Primary); ...,Asset Management and Custody Banks,Financials,Africa / Middle East,Jordan,-,16.500,3.12,3.03,-,AL-AMAL FINANCIAL INVESTMENTS CO.
3001,"As-me ESTELLE CO., LTD. (TSE:7872)",TSE:7872,Accessories (Primary); Apparel (Primary); Appa...,"Apparel, Accessories and Luxury Goods",Consumer Discretionary,Asia / Pacific,Japan,-,62.900,302.6,186.1,www.as-estelle.co.jp,"AS-ME ESTELLE CO., LTD."
3006,"ASAHI CO.,LTD. (TSE:3333)",TSE:3333,Consumer Discretionary (Primary); Leisure Equi...,Specialty Stores,Consumer Discretionary,Asia / Pacific,Japan,-,380.200,429.5,211,www.cb-asahi.jp,"ASAHI CO.,LTD."
3398,ATCO Ltd. (TSX:ACO.X),TSX:ACO.X,Multi-Utilities (Primary); Multi-Utilities (Pr...,Multi-Utilities,Utilities,United States and Canada,Canada,Alberta,3719.800,3174.7,2026.5,www.atco.com,ATCO LTD.
3408,"Aten International CO., Ltd. (TSEC:6277)",TSEC:6277,Computer Components (Primary); Information Tec...,"Technology Hardware, Storage and Peripherals",Information Technology,Asia / Pacific,Taiwan,-,303.100,153.7,90,www.aten.com.tw,"ATEN INTERNATIONAL CO., LTD."
4453,"BARUN Technology CO.,LTD. (KOSDAQ:A029480)",KOSDAQ:A029480,Creative Design Consulting (Primary); IMAC (In...,IT Consulting and Other Services,Information Technology,Asia / Pacific,South Korea,-,40.400,15,3.25,www.kdccorp.co.kr,"BARUN TECHNOLOGY CO.,LTD."
4771,"Belc CO., LTD. (TSE:9974)",TSE:9974,Consumer Staples (Primary); Food and Staples R...,Food Retail,Consumer Staples,Asia / Pacific,Japan,Saitama,824.600,1604.2,423.7,www.belc.co.jp,"BELC CO., LTD."
6926,"CAVE Interactive CO.,LTD. (JASDAQ:3760)",JASDAQ:3760,Information Technology (Primary); Internet Sof...,Internet Software and Services,Information Technology,Asia / Pacific,Japan,-,43.900,19.5,11.3,www.cave.co.jp,"CAVE INTERACTIVE CO.,LTD."
7244,"Cerespo CO., Ltd. (JASDAQ:9625)",JASDAQ:9625,Commercial and Professional Services (Primary)...,Diversified Support Services,Industrials,Asia / Pacific,Japan,-,19.800,95.6,29.5,www.cerespo.co.jp,"CERESPO CO., LTD."
7610,"China Aerospace Times Electronics CO., LTD. (S...",SHSE:600879,Aerospace and Defense (Primary); Aerospace and...,Aerospace and Defense,Industrials,Asia / Pacific,China,Hubei Province,2071.000,882.5,180,www.catec-ltd.cn,"CHINA AEROSPACE TIMES ELECTRONICS CO., LTD."


In [34]:
Merged[(Merged.country_code=='USA') & (Merged.status.isin(['ipo'])) & (pd.isnull(Merged.Market_Cap))]

,domain,country_code,state_code,region,city,status,short_description,category_list,funding_rounds,funding_total_usd,uuid,company_name_CB,company_name_normalized,company_name_CIQ,Headquarters_Country,State_Region,Market_Cap,IndustryClassifications


In [35]:
Funded_Companies[(Funded_Companies.country_code=='USA') & (Funded_Companies.status.isin(['ipo'])) ]

,domain,country_code,state_code,region,city,status,short_description,category_list,funding_rounds,funding_total_usd,uuid,company_name,company_name_mod
30530,1-page.com,USA,CA,SF Bay Area,San Francisco,ipo,1-Page (1PG:ASX) provides large enterprises wi...,Career Planning|Human Resources|Internet|Recru...,5,2.140000e+07,f6d14df3-69d3-fef5-824d-f93814a102ed,1-Page,1-PAGE
30560,1347capital.com,USA,IL,Chicago,Itasca,ipo,1347 Capital is a wholly owned subsidiary of K...,Banking,0,NaN,36280f78-ee16-d592-252d-ca66b7838dd5,1347 Capital,1347 CAPITAL
30561,1347pih.com,USA,FL,Tampa,Tampa,ipo,The Company’s insurance offerings currently in...,Finance,0,NaN,c95da493-cafe-7268-fc34-a667fba99673,1347 Property Insurance Holdings,1347 PROPERTY INSURANCE HOLDINGS
30565,NaN,USA,IL,Chicago,Chicago,ipo,broadcast oriented websites,Broadcasting,0,NaN,f81167a5-68df-2a2a-3883-93bb70e79208,141 CAPITAL,141 CAPITAL
30587,1cbank.com,USA,CA,Los Angeles,Los Angeles,ipo,1st Century Bank understands the value of powe...,Banking,0,NaN,d1fad38e-6113-81d9-6283-8b1875f044ed,1st Century Bank,1ST CENTURY BANK
30590,1stconstitution.com,USA,NJ,Newark,Cranbury,ipo,1ST Constitution Bancorp is a diversified fina...,Banking,0,NaN,090426bf-8518-f426-b024-737f8ab090ce,1st Constitution Bank,1ST CONSTITUTION BANK
30592,NaN,USA,NC,Greensboro,Burlington,ipo,1st State Bancorp will engage primarily in the...,Banking,0,NaN,1f2654d9-1b7c-7674-5fc3-aadb30a400e7,1st state bancorp,1ST STATE BANCORP
30605,21cf.com,USA,NY,New York City,New York,ipo,Mass Media Production,News|Photography|Product Design,0,NaN,ddfe6b91-4541-7f5c-7d6a-724f75c98088,21st Century Fox,21ST CENTURY FOX
30609,xxiicentury.com,USA,NY,NY - Other,Clarence,ipo,22nd Century Group is a plant biotechnology co...,Biotechnology,6,1.503475e+07,879f5bee-5780-e20f-77e4-f8cfc7567832,22nd Century Group,22ND CENTURY GROUP
30629,2dayembroidery.com,USA,NY,New York City,New York,ipo,"Do all kind of Embroidery, Digitizing and Grap...",Fashion|Printing,0,NaN,0532c005-593d-a630-504b-9e81e5fda14c,2Day Embroidery&Graphics,2DAY EMBROIDERY&GRAPHICS


In [24]:
len(Funded_Companies)

98182

In [36]:
Merged[Merged.company_name_normalized=='APPLE']

,domain,country_code,state_code,region,city,status,short_description,category_list,funding_rounds,funding_total_usd,uuid,company_name_CB,company_name_normalized,company_name_CIQ,Headquarters_Country,State_Region,Market_Cap,IndustryClassifications
909,apple.com,USA,CA,SF Bay Area,Cupertino,ipo,Apple is a multinational corporation that desi...,Consumer Electronics|Electronics|Hardware|Reta...,1,250000.0,7063d087-96b8-2cc1-ee88-c221288acc2a,Apple,APPLE,Apple Inc. (NasdaqGS:AAPL),United States,California,517945.3,Computer Components (Primary); Data Storage (P...


In [40]:
capiq_companies.dtypes

company_name                object
Ticker                      object
IndustryClassifications     object
Primary_Industry            object
Primary_Sector              object
Geographic_Region           object
Headquarters_Country        object
State_Region                object
Market_Cap                 float64
Total_Rev                   object
Gross_Profit                object
Website                     object
company_name_mod            object
Join                         int64
dtype: object

In [60]:
capiq_companies_Q = capiq_companies[capiq_companies.company_name_mod.str[0] =='Q'].copy()

In [61]:
Funded_Companies_Q = Funded_Companies[Funded_Companies.company_name_mod.str[0] =='Q'].copy()

In [62]:
len(capiq_companies_Q) * len(Funded_Companies_Q)

202540

In [63]:
Funded_Companies_Q['Join'] =1
capiq_companies_Q['Join'] =1

In [64]:
FullMerge_Q = pd.merge(Funded_Companies_Q,capiq_companies_Q, left_on='Join',right_on='Join')

In [66]:
len(FullMerge_Q)

202540

In [85]:
FullMerge_Q['Fuzz'] = FullMerge_Q.apply(lambda x: fuzz.ratio(x['company_name_mod_x'], x['company_name_mod_y']) , axis=1)

In [107]:
FullMerge_Q.sort_values(['company_name_mod_x','Fuzz'],ascending=False, inplace=True)

In [108]:
TopMatch = FullMerge_Q.groupby('company_name_mod_x',as_index=False).first()

In [114]:
TopMatch[['company_name_mod_x','company_name_mod_y','Fuzz']][TopMatch.Fuzz>85]

,company_name_mod_x,company_name_mod_y,Fuzz
5,Q HOLDINGS,Q2 HOLDINGS,95
13,Q-LINK TECHNOLOGIES,QLIK TECHNOLOGIES,94
32,QAD,QAD,100
36,QAHR,QHR,86
50,QBC HOLDINGS,QC HOLDINGS,96
54,QBINTERNATIONAL,QST INTERNATIONAL,88
61,QC HOLDINGS,QC HOLDINGS,100
64,QCR HOLDINGS,QCR HOLDINGS,100
82,QEP RESOURCES,QEP RESOURCES,100
91,QHB HOLDINGS,Q2 HOLDINGS,87


In [113]:
TopMatch.describe()

,funding_rounds,funding_total_usd,Join,Market_Cap,Fuzz
count,810.000000,5.510000e+02,810.0,810.000000,810.000000
mean,1.469136,2.301694e+07,1.0,1240.156795,65.834568
std,1.569852,1.031544e+08,0.0,5006.847907,12.253999
min,0.000000,4.000000e+03,1.0,0.008000,38.000000
25%,1.000000,2.620000e+05,1.0,32.200000,57.000000
50%,1.000000,2.089000e+06,1.0,139.000000,63.000000
75%,2.000000,1.200000e+07,1.0,890.800000,73.000000
max,18.000000,1.900000e+09,1.0,77544.000000,100.000000


In [200]:
Merged.to_excel('Capiq_Metrics.xlsx',index=False)

In [227]:
IndvidIndustries = Merged['IndustryClassifications'].str.split(';', expand=True, n=50)

In [240]:
a = Merged.IndustryClassifications.str.split(";").apply(pd.Series, 1).stack()
a.index = a.index.droplevel(-1)

a = a.to_frame()
a.columns=['Industries']

In [241]:
len(a)

48916

In [242]:
a['index'] = a.index

In [244]:
a.drop_duplicates(inplace=True)

In [264]:
PrimaryIndustries = a[a.Industries.str.contains('Primary')].copy()

In [265]:
PrimaryIndustries.rename(columns={'Industries':'Primary_Industries'},inplace=True)

In [259]:
SecondaryIndustries = a[~a['Industries'].str.contains('Primary')]

In [266]:
SecondaryIndustries.rename(columns={'Industries':'Secondary_Industries'},inplace=True)

In [272]:
Merged_Ind = pd.merge(Merged, PrimaryIndustries, left_index=True, right_index=True)

In [279]:
Merged_Ind.head()

,domain,country_code,state_code,region,city,status,short_description,category_list,funding_rounds,funding_total_usd,uuid,company_name_x,company_name_mod,company_name_y,Headquarters_Country,State_Region,Market_Cap,IndustryClassifications,Primary_Industries,index
0,nextmart.ae,ARE,NaN,Dubai,Dubai,operating,FoodOnClick for Groceries,E-Commerce,1,30000.0,d3e82a0a-a102-86b4-3c5b-198c2035627b,NextMart,NEXTMART,NextMart Inc. (OTCPK:NXMR),China,-,0.067,Advertising (Primary); Consumer Discretionary ...,Advertising (Primary),0
0,nextmart.ae,ARE,NaN,Dubai,Dubai,operating,FoodOnClick for Groceries,E-Commerce,1,30000.0,d3e82a0a-a102-86b4-3c5b-198c2035627b,NextMart,NEXTMART,NextMart Inc. (OTCPK:NXMR),China,-,0.067,Advertising (Primary); Consumer Discretionary ...,Consumer Discretionary (Primary),0
0,nextmart.ae,ARE,NaN,Dubai,Dubai,operating,FoodOnClick for Groceries,E-Commerce,1,30000.0,d3e82a0a-a102-86b4-3c5b-198c2035627b,NextMart,NEXTMART,NextMart Inc. (OTCPK:NXMR),China,-,0.067,Advertising (Primary); Consumer Discretionary ...,Media (Primary),0
1,redingtongulf.com,ARE,NaN,Dubai,Dubai,operating,Redington operates a website that provides inf...,Hardware|Software,1,65000000.0,99da02e6-81a4-f0af-1465-113774add600,Redington,REDINGTON,Redington (India) Ltd. (BSE:532805),India,Tamil Nadu,662.800,Communications Equipment Distribution (Primary...,Communications Equipment Distribution (Primary),1
1,redingtongulf.com,ARE,NaN,Dubai,Dubai,operating,Redington operates a website that provides inf...,Hardware|Software,1,65000000.0,99da02e6-81a4-f0af-1465-113774add600,Redington,REDINGTON,Redington (India) Ltd. (BSE:532805),India,Tamil Nadu,662.800,Communications Equipment Distribution (Primary...,"Electronic Equipment, Instruments and Compone...",1


In [283]:
Merged_Ind.pivot(columns='Primary_Industries')

domain  \
Primary_Industries  Accounting, Auditing and Taxation Services (Primary)   
0                                                                 NaN      
1                                                                 NaN      
2                                                                 NaN      
3                                                                 NaN      
4                                                                 NaN      
5                                                                 NaN      
6                                                                 NaN      
7                                                                 NaN      
8                                                                 NaN      
9                                                                 NaN      
10                                                                NaN      
11                                                                NaN      
12                                                                NaN      
13                                                                NaN      
14                                                                NaN      
15                                                                NaN      
16                                                                NaN      
17                                                                NaN      
18                                                                NaN      
19                                                                NaN      
20                                                                NaN      
21                                                                NaN      
22                                                                NaN      
23                                                                NaN      
24                                                                NaN      
25                                                                NaN      
26                                                                NaN      
27                                                                NaN      
28                                                                NaN      
29                                                                NaN      
...                                                               ...      
3624                                                              NaN      
3625                                                              NaN      
3626                                                              NaN      
3627                                                              NaN      
3628                                                              NaN      
3629                                                              NaN      
3630                                                              NaN      
3631                                                              NaN      
3632                                                              NaN      
3633                                                              NaN      
3634                                                              NaN      
3635                                                              NaN      
3636                                                              NaN      
3637                                                              NaN      
3638                                                              NaN      
3639                                                              NaN      
3640                                                              NaN      
3641                                                              NaN      
3642                                                              NaN      
3643                                                              NaN      
3644                                  

In [273]:
Merged_Ind.head(30)

,domain,country_code,state_code,region,city,status,short_description,category_list,funding_rounds,funding_total_usd,uuid,company_name_x,company_name_mod,company_name_y,Headquarters_Country,State_Region,Market_Cap,IndustryClassifications,Primary_Industries,index
0,nextmart.ae,ARE,NaN,Dubai,Dubai,operating,FoodOnClick for Groceries,E-Commerce,1,30000.0,d3e82a0a-a102-86b4-3c5b-198c2035627b,NextMart,NEXTMART,NextMart Inc. (OTCPK:NXMR),China,-,0.067,Advertising (Primary); Consumer Discretionary ...,Advertising (Primary),0
0,nextmart.ae,ARE,NaN,Dubai,Dubai,operating,FoodOnClick for Groceries,E-Commerce,1,30000.0,d3e82a0a-a102-86b4-3c5b-198c2035627b,NextMart,NEXTMART,NextMart Inc. (OTCPK:NXMR),China,-,0.067,Advertising (Primary); Consumer Discretionary ...,Consumer Discretionary (Primary),0
0,nextmart.ae,ARE,NaN,Dubai,Dubai,operating,FoodOnClick for Groceries,E-Commerce,1,30000.0,d3e82a0a-a102-86b4-3c5b-198c2035627b,NextMart,NEXTMART,NextMart Inc. (OTCPK:NXMR),China,-,0.067,Advertising (Primary); Consumer Discretionary ...,Media (Primary),0
1,redingtongulf.com,ARE,NaN,Dubai,Dubai,operating,Redington operates a website that provides inf...,Hardware|Software,1,65000000.0,99da02e6-81a4-f0af-1465-113774add600,Redington,REDINGTON,Redington (India) Ltd. (BSE:532805),India,Tamil Nadu,662.800,Communications Equipment Distribution (Primary...,Communications Equipment Distribution (Primary),1
1,redingtongulf.com,ARE,NaN,Dubai,Dubai,operating,Redington operates a website that provides inf...,Hardware|Software,1,65000000.0,99da02e6-81a4-f0af-1465-113774add600,Redington,REDINGTON,Redington (India) Ltd. (BSE:532805),India,Tamil Nadu,662.800,Communications Equipment Distribution (Primary...,"Electronic Equipment, Instruments and Compone...",1
1,redingtongulf.com,ARE,NaN,Dubai,Dubai,operating,Redington operates a website that provides inf...,Hardware|Software,1,65000000.0,99da02e6-81a4-f0af-1465-113774add600,Redington,REDINGTON,Redington (India) Ltd. (BSE:532805),India,Tamil Nadu,662.800,Communications Equipment Distribution (Primary...,Information Technology (Primary),1
1,redingtongulf.com,ARE,NaN,Dubai,Dubai,operating,Redington operates a website that provides inf...,Hardware|Software,1,65000000.0,99da02e6-81a4-f0af-1465-113774add600,Redington,REDINGTON,Redington (India) Ltd. (BSE:532805),India,Tamil Nadu,662.800,Communications Equipment Distribution (Primary...,Software Distribution (Primary),1
1,redingtongulf.com,ARE,NaN,Dubai,Dubai,operating,Redington operates a website that provides inf...,Hardware|Software,1,65000000.0,99da02e6-81a4-f0af-1465-113774add600,Redington,REDINGTON,Redington (India) Ltd. (BSE:532805),India,Tamil Nadu,662.800,Communications Equipment Distribution (Primary...,Technology Distributors (Primary),1
1,redingtongulf.com,ARE,NaN,Dubai,Dubai,operating,Redington operates a website that provides inf...,Hardware|Software,1,65000000.0,99da02e6-81a4-f0af-1465-113774add600,Redington,REDINGTON,Redington (India) Ltd. (BSE:532805),India,Tamil Nadu,662.800,Communications Equipment Distribution (Primary...,Technology Hardware and Equipment (Primary),1
1,redingtongulf.com,ARE,NaN,Dubai,Dubai,operating,Redington operates a website that provides inf...,Hardware|Software,1,65000000.0,99da02e6-81a4-f0af-1465-113774add600,Redington,REDINGTON,Redington (India) Ltd. (BSE:532805),India,Tamil Nadu,662.800,Communications Equipment Distribution (Primary...,Technology Hardware and Equipment Distributio...,1


In [230]:
IndvidIndustries.tail()

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
322,Accessories (Primary),Apparel (Primary),"Apparel, Accessories and Luxury Goods (Primary)",Belts and Suspenders (Primary),Consumer Discretionary (Primary),Consumer Durables and Apparel (Primary),Luggage and Handbags (Primary),Men's Apparel (Primary),Men's Coats and Jackets (Primary),Men's Shirts and Sweaters (Primary),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3522,Information Technology (Primary),Internet Security Software (Primary),Internet Software (Primary),Internet Software and Services (Primary),Internet Software and Services (Primary),Software and Services (Primary),Website Infrastructure Software (Primary),None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3523,Healthcare (Primary),Pharmaceutical Products (Primary),Pharmaceuticals (Primary),Pharmaceuticals (Primary),"Pharmaceuticals, Biotechnology and Life Scien...",None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3653,Healthcare (Primary),Healthcare Equipment (Primary),Healthcare Equipment and Services (Primary),Healthcare Equipment and Supplies (Primary),"Medical Testing, Analyzing, and Diagnostic Eq...",Therapeutic Devices (Primary),Healthcare Distributors,Healthcare Equipment Distribution,Healthcare Providers and Services,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3524,Computer Games (Primary),Entertainment Software (Primary),Home Entertainment Software (Primary),Information Technology (Primary),Software (Primary),Software and Services (Primary),Advertising,Casinos and Gaming,Consumer Discretionary,Consumer Services,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [228]:
IndvidIndustries[IndvidIndustries.columns[40:]].describe()

,40,41,42,43,44,45,46,47,48,49,50
count,61,59,54,54,51,47,41,40,39,38,35
unique,52,53,46,48,44,42,33,36,33,33,33
top,Software and Services,Outsourced Technology Manufacturing Services,Machinery,Information Technology,Software,Software and Services,Real Estate Management and Development,Electric Utilities,Retailing,Specialty Retail,"Technology Hardware, Storage and Peripherals;..."
freq,4,2,3,2,3,3,2,2,4,2,2
